# Frequent Itemset Mining

Learning Objectives:

- Extract frequent patterns given a corpus of data.
- Find the rules which are interesting and non-obvious for a given domain.

In [59]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

df = pd.read_excel('Online Retail.xlsx')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [60]:
df2 = pd.read_excel('online_retail_II.xlsx')
df2.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


Question 1.1: Concatinate both dataframes to create a single dataframe. Remove any rows where InvoiceNo is Null and Quantity is Negative

In [97]:
frames = [df, df2]

data = pd.concat(frames)
data.dropna(subset=['InvoiceNo'],inplace=True)
data= data[data['Quantity']>0]
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Invoice,Price,Customer ID
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,NaN,NaN,NaN
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,NaN,NaN
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,NaN,NaN,NaN
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,NaN,NaN
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,NaN,NaN,NaN


Question 1.2: Filter the data by only transactions that happened in United Kingdom 

In [98]:
UK_Data = data[data['Country']=='United Kingdom']
UK_Data['Country'].value_counts()

United Kingdom    486286
Name: Country, dtype: int64

Question 1.3: What are the most popular 5 items?

In [99]:
item_freq = UK_Data['StockCode'].value_counts()

item_freq.head(5)

85123A    2174
85099B    1960
22423     1711
47566     1615
20725     1421
Name: StockCode, dtype: int64

In [92]:
item_freq.head(5).index.to_list()

['85123A', '85099B', 22423, 47566, 20725]

Question 1.4: Filter down the data to include transaction that contain the top 20 items

In [93]:
topitems = UK_Data['StockCode'].astype(str).isin(item_freq.head(5).index.to_list())

In [101]:
UK_Data.loc[topitems,'InvoiceNo']

0         536365
49        536373
66        536375
177       536386
220       536390
           ...  
540217    581472
540434    581486
540837    581492
541536    581498
541639    581538
Name: InvoiceNo, Length: 4134, dtype: object

In [102]:
InvoicesOfInterest = UK_Data['InvoiceNo'].isin(UK_Data.loc[topitems,'InvoiceNo'].to_list())

In [105]:
UK_Data_of_Interest = UK_Data.loc[InvoicesOfInterest,:]

Question 2.1: Consolidate the items into 1 transaction per row and each product one-hot encoded.

In [106]:
basket = (UK_Data_of_Interest
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

In [107]:
basket

Description,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC SWEETHEART SOAP DISH,ZINC SWEETHEART WIRE LETTER RACK,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL,ZINC TOP 2 DOOR WOODEN SHELF,ZINC WILLIE WINKIE CANDLE STICK,ZINC WIRE KITCHEN ORGANISER,ZINC WIRE SWEETHEART LETTER TRAY
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536365,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536373,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536375,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
536390,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
581472,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
581492,0.0,1.0,1.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Question 2.2: Convert all the values to 1 when values are greater than 0 and 0 when values are 0 or less.

In [111]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

Question 3.1: Apply [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) algorithm to generate frequent item sets that have a support of at least 7%

In [112]:
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

Question 3.2: Generate the association rules with their corresponding support, confidence and lift.

In [113]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ASSORTED COLOUR BIRD ORNAMENT),(WHITE HANGING HEART T-LIGHT HOLDER),0.110230,0.587801,0.083425,0.756824,1.287551,0.018631,1.695064
1,(WHITE HANGING HEART T-LIGHT HOLDER),(ASSORTED COLOUR BIRD ORNAMENT),0.587801,0.110230,0.083425,0.141926,1.287551,0.018631,1.036939
2,(WHITE HANGING HEART T-LIGHT HOLDER),(CANDLEHOLDER PINK HANGING HEART),0.587801,0.080689,0.072210,0.122848,1.522480,0.024781,1.048063
3,(CANDLEHOLDER PINK HANGING HEART),(WHITE HANGING HEART T-LIGHT HOLDER),0.080689,0.587801,0.072210,0.894915,1.522480,0.024781,3.922540
4,(JUMBO BAG RED RETROSPOT),(CHARLOTTE BAG PINK POLKADOT),0.530088,0.090536,0.071663,0.135191,1.493227,0.023671,1.051635


Question 4: Based on the above rules, identify what would be the opportunity of promoting one of the antecendents.